In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

import os
import csv

In [4]:
loc = ''
db_name = 'random_train.db'

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(loc+db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None

In [41]:
con = create_connection(db_name)

In [42]:
if con is not None:
    tag_data = pd.read_sql('SELECT Tags FROM data',con)
else :
    print('Conn error')

In [43]:
tag_data.head()

,Tags
0,iphone objective-c ios uiview uibutton
1,svg internet-explorer-9 raphael
2,validation spring-mvc internationalization cus...
3,windows java copy text
4,javascript jquery
